In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import timedelta

In [3]:
slurm = pd.read_csv("/mnt/research/CMSE495-SS24-ICER/slurm_usage/DID_FINAL_SLURM_OCT_2023.csv",delimiter="|", nrows=100000)


In [4]:
slurm = slurm.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])
slurm.head()

,JobID,User,Group,Submit,Start,End,Elapsed,State,Account,AssocID,...,CPUTimeRAW,ReqCPUS,AllocCPUS,ReqMem,MaxRSS,ReqNodes,NNodes,NodeList,ReqTRES,AllocTRES
0,31496544,user_679,group_121,2023-03-21T11:13:45,Unknown,Unknown,00:00:00,PENDING,account_017,assocID_489,...,0,28,0,21000M,NaN,1,1,None assigned,"billing=3192,cpu=28,gres/gpu=4,mem=21000M,node=1",NaN
1,31497932,user_679,group_121,2023-03-21T11:31:18,Unknown,Unknown,00:00:00,PENDING,account_017,assocID_489,...,0,28,0,21000M,NaN,1,1,None assigned,"billing=3192,cpu=28,gres/gpu=4,mem=21000M,node=1",NaN
2,31993628,user_105,group_114,2023-03-22T18:19:12,Unknown,Unknown,00:00:00,PENDING,account_017,assocID_661,...,0,12,0,150G,NaN,1,1,None assigned,"billing=23347,cpu=12,gres/gpu=8,mem=150G,node=1",NaN
3,39087660,user_652,group_054,2023-04-04T13:09:10,Unknown,Unknown,00:00:00,PENDING,account_017,assocID_557,...,0,640,0,20G,NaN,10,10,None assigned,"billing=3112,cpu=640,mem=20G,node=10",NaN
4,59062820,user_188,group_046,2023-05-08T09:58:20,2024-01-01T00:58:57,2024-01-01T00:59:06,00:00:09,COMPLETED,account_017,assocID_676,...,360,40,40,8G,NaN,1,1,skl-029,"billing=1245,cpu=40,mem=8G,node=1","billing=1245,cpu=40,mem=8G,node=1"


In [5]:
slurm.columns

Index(['JobID', 'User', 'Group', 'Submit', 'Start', 'End', 'Elapsed', 'State',
       'Account', 'AssocID', 'Partition', 'Timelimit', 'UserCPU', 'SystemCPU',
       'TotalCPU', 'CPUTime', 'CPUTimeRAW', 'ReqCPUS', 'AllocCPUS', 'ReqMem',
       'MaxRSS', 'ReqNodes', 'NNodes', 'NodeList', 'ReqTRES', 'AllocTRES'],
      dtype='object')

In [6]:
slurm.shape

(100000, 26)

# Preprocessing Data

In [7]:
# # convert Submit and Start to datetime, and filter rows with 'Unknown' start

# # Filter out rows where 'State' is "Cancelled" or Unknown
slurm = slurm[slurm['State'] != 'Cancelled']
slurm = slurm[slurm['Start']!= 'Unknown']

slurm['Submit'] = pd.to_datetime(slurm['Submit'])
slurm['Start'] = pd.to_datetime(slurm['Start'])

slurm.head(5)

,JobID,User,Group,Submit,Start,End,Elapsed,State,Account,AssocID,...,CPUTimeRAW,ReqCPUS,AllocCPUS,ReqMem,MaxRSS,ReqNodes,NNodes,NodeList,ReqTRES,AllocTRES
4,59062820,user_188,group_046,2023-05-08 09:58:20,2024-01-01 00:58:57,2024-01-01T00:59:06,00:00:09,COMPLETED,account_017,assocID_676,...,360,40,40,8G,NaN,1,1,skl-029,"billing=1245,cpu=40,mem=8G,node=1","billing=1245,cpu=40,mem=8G,node=1"
5,59062828,user_188,group_046,2023-05-08 09:58:20,2024-01-01 01:00:49,2024-01-01T01:00:55,00:00:06,COMPLETED,account_017,assocID_676,...,240,40,40,8G,NaN,1,1,skl-029,"billing=1245,cpu=40,mem=8G,node=1","billing=1245,cpu=40,mem=8G,node=1"
6,59062836,user_188,group_046,2023-05-08 09:58:20,2024-01-01 01:00:49,2024-01-01T01:00:55,00:00:06,COMPLETED,account_017,assocID_676,...,240,40,40,8G,NaN,1,1,skl-030,"billing=1245,cpu=40,mem=8G,node=1","billing=1245,cpu=40,mem=8G,node=1"
7,59062844,user_188,group_046,2023-05-08 09:58:21,2024-01-01 00:58:57,2024-01-01T00:59:06,00:00:09,COMPLETED,account_017,assocID_676,...,360,40,40,8G,NaN,1,1,amr-207,"billing=1245,cpu=40,mem=8G,node=1","billing=1245,cpu=40,mem=8G,node=1"
8,59062852,user_188,group_046,2023-05-08 09:58:21,2024-01-01 00:58:57,2024-01-01T00:59:06,00:00:09,COMPLETED,account_017,assocID_676,...,360,40,40,8G,NaN,1,1,amr-207,"billing=1245,cpu=40,mem=8G,node=1","billing=1245,cpu=40,mem=8G,node=1"


In [8]:
# remove user_258 and clean jobs data
def AggSLURMDat(dat):
    '''
    Aggregates all submitted jobs together, removing all batch/extern 
    jobs and including said information into a single job. Excludes
    jobs that do not have a clear '.batch' and '.extern' files

    args:

        dat - the slurm dataset 
    
    returns:

        out_df - the aggregated version of the slurm dataset
    '''
    
    job_list = dat["JobID"].value_counts().index

    out_df = pd.DataFrame(columns=dat.keys())

    for job in job_list:

        jdat = dat[dat["JobID"] == job]

        cpu_time_list = jdat["CPUTimeRAW"].value_counts()
        cpu_time_list = cpu_time_list[cpu_time_list == 2].index

        for cpu_time in cpu_time_list:

            ajob = jdat[jdat["CPUTimeRAW"] == cpu_time]

            batch_job = ajob[ajob["User"] == "user_258"]

            ag_job = ajob[ajob["User"] != "user_258"]

            if len(ag_job["User"]) == 0:
#                 print("Weird Job",ajob["JobID"])
#                 print("No aggregate job")
                continue
            
            if len(ag_job["User"]) == 2:
#                 print("Weird Job",ajob["JobID"])
#                 print("2 copies of aggregate job")
                continue

            assert len(ag_job["User"]) == 1, "New edge case discovered!"

            ag_job.loc[ag_job.index[0],"MaxRSS"] = batch_job["MaxRSS"].values[0]

            out_df = pd.concat([out_df,ag_job])

    return out_df

In [9]:
# test
slurm_cleaned = AggSLURMDat(slurm)
slurm_cleaned

,JobID,User,Group,Submit,Start,End,Elapsed,State,Account,AssocID,...,CPUTimeRAW,ReqCPUS,AllocCPUS,ReqMem,MaxRSS,ReqNodes,NNodes,NodeList,ReqTRES,AllocTRES
75766,91806820,user_316,group_009,2023-10-01 10:08:04,2023-10-01 10:08:30,2023-10-01T10:11:08,00:02:38,COMPLETED,account_017,assocID_554,...,158,1,1,10G,348380K,1,1,lac-351,"billing=1556,cpu=1,mem=10G,node=1","billing=1556,cpu=1,mem=10G,node=1"
75880,91806820,user_316,group_009,2023-10-01 10:08:04,2023-10-01 10:08:30,2023-10-01T10:09:18,00:00:48,FAILED,account_017,assocID_554,...,48,1,1,10G,8156K,1,1,lac-367,"billing=1556,cpu=1,mem=10G,node=1","billing=1556,cpu=1,mem=10G,node=1"
77026,91806820,user_316,group_009,2023-10-01 10:08:04,2023-10-01 10:08:31,2023-10-01T10:10:33,00:02:02,FAILED,account_017,assocID_554,...,122,1,1,10G,344968K,1,1,lac-295,"billing=1556,cpu=1,mem=10G,node=1","billing=1556,cpu=1,mem=10G,node=1"
77281,91806820,user_316,group_009,2023-10-01 10:08:04,2023-10-01 10:08:31,2023-10-01T10:11:31,00:03:00,COMPLETED,account_017,assocID_554,...,180,1,1,10G,489700K,1,1,lac-299,"billing=1556,cpu=1,mem=10G,node=1","billing=1556,cpu=1,mem=10G,node=1"
76963,91806820,user_316,group_009,2023-10-01 10:08:04,2023-10-01 10:08:31,2023-10-01T10:09:36,00:01:05,FAILED,account_017,assocID_554,...,65,1,1,10G,270400K,1,1,lac-294,"billing=1556,cpu=1,mem=10G,node=1","billing=1556,cpu=1,mem=10G,node=1"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29569,91740004,user_128,group_062,2023-10-01 00:37:59,2023-10-01 00:39:32,2023-10-01T00:46:35,00:07:03,COMPLETED,account_030,assocID_561,...,1692,4,4,8G,531208K,1,1,amr-162,"billing=1245,cpu=4,mem=8G,node=1","billing=1245,cpu=4,mem=8G,node=1"
29572,91740008,user_128,group_062,2023-10-01 00:37:59,2023-10-01 00:39:38,2023-10-01T00:46:42,00:07:04,COMPLETED,account_030,assocID_561,...,1696,4,4,8G,528908K,1,1,amr-162,"billing=1245,cpu=4,mem=8G,node=1","billing=1245,cpu=4,mem=8G,node=1"
29575,91740012,user_128,group_062,2023-10-01 00:38:00,2023-10-01 00:39:43,2023-10-01T00:45:59,00:06:16,COMPLETED,account_030,assocID_561,...,1504,4,4,8G,490392K,1,1,amr-162,"billing=1245,cpu=4,mem=8G,node=1","billing=1245,cpu=4,mem=8G,node=1"
29530,91739952,user_128,group_062,2023-10-01 00:37:49,2023-10-01 00:38:49,2023-10-01T00:50:17,00:11:28,COMPLETED,account_030,assocID_561,...,2752,4,4,8G,747872K,1,1,amr-162,"billing=1245,cpu=4,mem=8G,node=1","billing=1245,cpu=4,mem=8G,node=1"


 # Feature Engineering of Cleaned SLURM 
 - After Mapping user258 to find underutilization in Memery
 
 1. Convert memory units to all be in MB columns:ReqMem, MaxRSS
 2. Convert Timelimit and Elapsed columns from object to datetime

In [10]:
slurm_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15689 entries, 75766 to 29500
Data columns (total 26 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   JobID       15689 non-null  object        
 1   User        15689 non-null  object        
 2   Group       15689 non-null  object        
 3   Submit      15689 non-null  datetime64[ns]
 4   Start       15689 non-null  datetime64[ns]
 5   End         15689 non-null  object        
 6   Elapsed     15689 non-null  object        
 7   State       15689 non-null  object        
 8   Account     15689 non-null  object        
 9   AssocID     15689 non-null  object        
 10  Partition   15689 non-null  object        
 11  Timelimit   15689 non-null  object        
 12  UserCPU     15689 non-null  object        
 13  SystemCPU   15689 non-null  object        
 14  TotalCPU    15689 non-null  object        
 15  CPUTime     15689 non-null  object        
 16  CPUTimeRAW  15689 

In [11]:
# remove na rows in MaxRSS after cleaning data
slurm_cleaned= slurm_cleaned.dropna(subset=['MaxRSS'])
slurm_cleaned

,JobID,User,Group,Submit,Start,End,Elapsed,State,Account,AssocID,...,CPUTimeRAW,ReqCPUS,AllocCPUS,ReqMem,MaxRSS,ReqNodes,NNodes,NodeList,ReqTRES,AllocTRES
75766,91806820,user_316,group_009,2023-10-01 10:08:04,2023-10-01 10:08:30,2023-10-01T10:11:08,00:02:38,COMPLETED,account_017,assocID_554,...,158,1,1,10G,348380K,1,1,lac-351,"billing=1556,cpu=1,mem=10G,node=1","billing=1556,cpu=1,mem=10G,node=1"
75880,91806820,user_316,group_009,2023-10-01 10:08:04,2023-10-01 10:08:30,2023-10-01T10:09:18,00:00:48,FAILED,account_017,assocID_554,...,48,1,1,10G,8156K,1,1,lac-367,"billing=1556,cpu=1,mem=10G,node=1","billing=1556,cpu=1,mem=10G,node=1"
77026,91806820,user_316,group_009,2023-10-01 10:08:04,2023-10-01 10:08:31,2023-10-01T10:10:33,00:02:02,FAILED,account_017,assocID_554,...,122,1,1,10G,344968K,1,1,lac-295,"billing=1556,cpu=1,mem=10G,node=1","billing=1556,cpu=1,mem=10G,node=1"
77281,91806820,user_316,group_009,2023-10-01 10:08:04,2023-10-01 10:08:31,2023-10-01T10:11:31,00:03:00,COMPLETED,account_017,assocID_554,...,180,1,1,10G,489700K,1,1,lac-299,"billing=1556,cpu=1,mem=10G,node=1","billing=1556,cpu=1,mem=10G,node=1"
76963,91806820,user_316,group_009,2023-10-01 10:08:04,2023-10-01 10:08:31,2023-10-01T10:09:36,00:01:05,FAILED,account_017,assocID_554,...,65,1,1,10G,270400K,1,1,lac-294,"billing=1556,cpu=1,mem=10G,node=1","billing=1556,cpu=1,mem=10G,node=1"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29569,91740004,user_128,group_062,2023-10-01 00:37:59,2023-10-01 00:39:32,2023-10-01T00:46:35,00:07:03,COMPLETED,account_030,assocID_561,...,1692,4,4,8G,531208K,1,1,amr-162,"billing=1245,cpu=4,mem=8G,node=1","billing=1245,cpu=4,mem=8G,node=1"
29572,91740008,user_128,group_062,2023-10-01 00:37:59,2023-10-01 00:39:38,2023-10-01T00:46:42,00:07:04,COMPLETED,account_030,assocID_561,...,1696,4,4,8G,528908K,1,1,amr-162,"billing=1245,cpu=4,mem=8G,node=1","billing=1245,cpu=4,mem=8G,node=1"
29575,91740012,user_128,group_062,2023-10-01 00:38:00,2023-10-01 00:39:43,2023-10-01T00:45:59,00:06:16,COMPLETED,account_030,assocID_561,...,1504,4,4,8G,490392K,1,1,amr-162,"billing=1245,cpu=4,mem=8G,node=1","billing=1245,cpu=4,mem=8G,node=1"
29530,91739952,user_128,group_062,2023-10-01 00:37:49,2023-10-01 00:38:49,2023-10-01T00:50:17,00:11:28,COMPLETED,account_030,assocID_561,...,2752,4,4,8G,747872K,1,1,amr-162,"billing=1245,cpu=4,mem=8G,node=1","billing=1245,cpu=4,mem=8G,node=1"


In [12]:
# no null values
# slurm_cleaned.info()

print(set(slurm_cleaned.ReqMem))
print(set(slurm_cleaned.MaxRSS))

{'16192M', '8G', '2016G', '24G', '96000M', '285G', '50000M', '120G', '505202M', '750M', '490G', '1200G', '50G', '28G', '128G', '2.50G', '75G', '160G', '80G', '512M', '800G', '5G', '512G', '32G', '1T', '110G', '2T', '60G', '900G', '300G', '44G', '480G', '10G', '164G', '384G', '4G', '16G', '96G', '100G', '192G', '20G', '2500G', '750G', '4500M', '48000M', '320G', '200G', '18G', '36G', '70G', '8000M', '3000M', '30G', '500G', '2G', '600G', '40G', '64G'}
{'14455384K', '4793988K', '128748K', '1282344K', '65418380K', '772700K', '52540K', '1296252K', '1281196K', '2167884K', '4046780K', '120623220K', '1277184K', '804712K', '1288152K', '65432148K', '31480K', '3780292K', '419768K', '130062024K', '23888432K', '59336K', '51452K', '7408K', '3249064K', '407784K', '2026592K', '376660K', '419352K', '389088K', '1275812K', '710688K', '23762168K', '911736K', '119528K', '1273324K', '65431944K', '23830000K', '83328K', '124248K', '1279684K', '59948K', '23661984K', '1275504K', '65435924K', '65455960K', '800904

In [13]:
# import warnings
# warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

# # convert ReqMem to a uniform measurement (assuming 'M' for MB and 'G' for GB and 'K' for KB)
def convert_memory(mem_str):
    '''
    Convert memory units to MegaBytes unit float.
    '''
    if mem_str.endswith('M'):
        return float(mem_str[:-1]) # remove 'M' and convert to float
    elif mem_str.endswith('K'):
        return float(mem_str[:-1]) / 1000
    elif mem_str.endswith('G'):
        return float(mem_str[:-1]) * 1e3  # convert MB to KB
    elif mem_str.endswith('T'):
        return float(mem_str[:-1]) * 1e6 # convert MB to T
    

In [14]:
slurm_cleaned['ReqMem_MB'] = slurm_cleaned['ReqMem'].apply(convert_memory)
slurm_cleaned['MaxRSS_MB'] = slurm_cleaned['MaxRSS'].apply(convert_memory)

slurm_cleaned

/tmp/local/32730612/ipykernel_248495/1897884236.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  slurm_cleaned['ReqMem_MB'] = slurm_cleaned['ReqMem'].apply(convert_memory)
/tmp/local/32730612/ipykernel_248495/1897884236.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  slurm_cleaned['MaxRSS_MB'] = slurm_cleaned['MaxRSS'].apply(convert_memory)


,JobID,User,Group,Submit,Start,End,Elapsed,State,Account,AssocID,...,AllocCPUS,ReqMem,MaxRSS,ReqNodes,NNodes,NodeList,ReqTRES,AllocTRES,ReqMem_MB,MaxRSS_MB
75766,91806820,user_316,group_009,2023-10-01 10:08:04,2023-10-01 10:08:30,2023-10-01T10:11:08,00:02:38,COMPLETED,account_017,assocID_554,...,1,10G,348380K,1,1,lac-351,"billing=1556,cpu=1,mem=10G,node=1","billing=1556,cpu=1,mem=10G,node=1",10000.0,348.380
75880,91806820,user_316,group_009,2023-10-01 10:08:04,2023-10-01 10:08:30,2023-10-01T10:09:18,00:00:48,FAILED,account_017,assocID_554,...,1,10G,8156K,1,1,lac-367,"billing=1556,cpu=1,mem=10G,node=1","billing=1556,cpu=1,mem=10G,node=1",10000.0,8.156
77026,91806820,user_316,group_009,2023-10-01 10:08:04,2023-10-01 10:08:31,2023-10-01T10:10:33,00:02:02,FAILED,account_017,assocID_554,...,1,10G,344968K,1,1,lac-295,"billing=1556,cpu=1,mem=10G,node=1","billing=1556,cpu=1,mem=10G,node=1",10000.0,344.968
77281,91806820,user_316,group_009,2023-10-01 10:08:04,2023-10-01 10:08:31,2023-10-01T10:11:31,00:03:00,COMPLETED,account_017,assocID_554,...,1,10G,489700K,1,1,lac-299,"billing=1556,cpu=1,mem=10G,node=1","billing=1556,cpu=1,mem=10G,node=1",10000.0,489.700
76963,91806820,user_316,group_009,2023-10-01 10:08:04,2023-10-01 10:08:31,2023-10-01T10:09:36,00:01:05,FAILED,account_017,assocID_554,...,1,10G,270400K,1,1,lac-294,"billing=1556,cpu=1,mem=10G,node=1","billing=1556,cpu=1,mem=10G,node=1",10000.0,270.400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29569,91740004,user_128,group_062,2023-10-01 00:37:59,2023-10-01 00:39:32,2023-10-01T00:46:35,00:07:03,COMPLETED,account_030,assocID_561,...,4,8G,531208K,1,1,amr-162,"billing=1245,cpu=4,mem=8G,node=1","billing=1245,cpu=4,mem=8G,node=1",8000.0,531.208
29572,91740008,user_128,group_062,2023-10-01 00:37:59,2023-10-01 00:39:38,2023-10-01T00:46:42,00:07:04,COMPLETED,account_030,assocID_561,...,4,8G,528908K,1,1,amr-162,"billing=1245,cpu=4,mem=8G,node=1","billing=1245,cpu=4,mem=8G,node=1",8000.0,528.908
29575,91740012,user_128,group_062,2023-10-01 00:38:00,2023-10-01 00:39:43,2023-10-01T00:45:59,00:06:16,COMPLETED,account_030,assocID_561,...,4,8G,490392K,1,1,amr-162,"billing=1245,cpu=4,mem=8G,node=1","billing=1245,cpu=4,mem=8G,node=1",8000.0,490.392
29530,91739952,user_128,group_062,2023-10-01 00:37:49,2023-10-01 00:38:49,2023-10-01T00:50:17,00:11:28,COMPLETED,account_030,assocID_561,...,4,8G,747872K,1,1,amr-162,"billing=1245,cpu=4,mem=8G,node=1","billing=1245,cpu=4,mem=8G,node=1",8000.0,747.872


In [15]:
# import warnings
# warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

# Function to parse time data columns: TimeLimit and Elapsed
def parse_time_string(time_str):
    """Convert a time string into a timedelta object."""
    days = 0
    if '-' in time_str:
        days, time_str = time_str.split('-')
        days = int(days)

    parts = time_str.split(':')
    hours, minutes, seconds = map(int, parts) if len(parts) == 3 else (int(parts[0]), int(parts[1]), 0)
    return timedelta(days=days, hours=hours, minutes=minutes, seconds=seconds)

# Convert Timelimit and Elapsed to timedeltas
slurm_cleaned['Timelimit'] = slurm_cleaned['Timelimit'].apply(parse_time_string)
slurm_cleaned['Elapsed'] = slurm_cleaned['Elapsed'].apply(parse_time_string)

/tmp/local/32730612/ipykernel_248495/34890253.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  slurm_cleaned['Timelimit'] = slurm_cleaned['Timelimit'].apply(parse_time_string)
/tmp/local/32730612/ipykernel_248495/34890253.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  slurm_cleaned['Elapsed'] = slurm_cleaned['Elapsed'].apply(parse_time_string)


In [22]:
slurm_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14271 entries, 75766 to 29500
Data columns (total 32 columns):
 #   Column                    Non-Null Count  Dtype          
---  ------                    --------------  -----          
 0   JobID                     14271 non-null  object         
 1   User                      14271 non-null  object         
 2   Group                     14271 non-null  object         
 3   Submit                    14271 non-null  datetime64[ns] 
 4   Start                     14271 non-null  datetime64[ns] 
 5   End                       14271 non-null  object         
 6   Elapsed                   14271 non-null  timedelta64[ns]
 7   State                     14271 non-null  object         
 8   Account                   14271 non-null  object         
 9   AssocID                   14271 non-null  object         
 10  Partition                 14271 non-null  object         
 11  Timelimit                 14271 non-null  timedelta64[ns]
 12  

# Function to list users that underutilize CPU, Time, and Nodes 

In [17]:
# The higher the number is the more the user underutilizes

def FindUnterutilizerSLURM(data, time_threshold, cpu_threshold, nodes_threshold, memory_threshold):
    """
    Identifies SLURM job records that underutilize allocated resources compared to requested ones. Each
    resource (time, CPU, nodes, memory) is evaluated against its threshold; jobs that exceed any threshold
    are included in the output. The function returns a DataFrame listing underutilizing jobs along with
    the amount by which they underutilize each resource.

    Parameters:
    :param data: DataFrame with SLURM job records.
    :param time_threshold: Minimum hours a job must underutilize its allocated time to be included.
    :param cpu_threshold: Minimum number of CPUs a job must underutilize to be included.
    :param nodes_threshold: Minimum number of nodes a job must underutilize to be included.
    :param memory_threshold: Minimum fraction (decimal, not percentage) of memory underutilization for inclusion.
    
    Returns:
    DataFrame with columns for 'User', 'JobID', 'Group', 'State', 'Account', and underutilized resources.
    """
    
    # Calculate underutilized resources
    data['UnderUtilizeCPU'] = data.ReqCPUS - data.AllocCPUS    
    data['UnderUtilizeNodes'] = data.ReqNodes - data.NNodes  
    data['UnderUtilizeTime (Hours)'] = (data['Timelimit'] - data['Elapsed']) / pd.Timedelta(hours=1)   
    data['UnderUtilizeMemory'] = ((data['ReqMem_MB'] - data['MaxRSS_MB']) / data['ReqMem_MB'])
    
    # Apply thresholds and filter rows
    filtered_data = data.loc[
        (data['UnderUtilizeCPU'] > cpu_threshold) |
        (data['UnderUtilizeNodes'] > nodes_threshold) |
        (data['UnderUtilizeTime (Hours)'] > time_threshold) |
        (data['UnderUtilizeMemory'] > memory_threshold)
    ]
    
    return filtered_data[['User', 'JobID', 'Group', 'State', 'Account', 'UnderUtilizeTime (Hours)', 'UnderUtilizeCPU', 'UnderUtilizeNodes', 'UnderUtilizeMemory']]


In [18]:
# Test on 100,000 rows
FindUnterutilizerSLURM(slurm_cleaned,time_threshold=10, cpu_threshold=0, nodes_threshold=3, memory_threshold=0.95)

/tmp/local/32730612/ipykernel_248495/2472876881.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['UnderUtilizeCPU'] = data.ReqCPUS - data.AllocCPUS
/tmp/local/32730612/ipykernel_248495/2472876881.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['UnderUtilizeNodes'] = data.ReqNodes - data.NNodes
/tmp/local/32730612/ipykernel_248495/2472876881.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

,User,JobID,Group,State,Account,UnderUtilizeTime (Hours),UnderUtilizeCPU,UnderUtilizeNodes,UnderUtilizeMemory
75766,user_316,91806820,group_009,COMPLETED,account_017,0.272778,0,0,0.965162
75880,user_316,91806820,group_009,FAILED,account_017,0.303333,0,0,0.999184
77026,user_316,91806820,group_009,FAILED,account_017,0.282778,0,0,0.965503
77281,user_316,91806820,group_009,COMPLETED,account_017,0.266667,0,0,0.951030
76963,user_316,91806820,group_009,FAILED,account_017,0.298611,0,0,0.972960
...,...,...,...,...,...,...,...,...,...
29569,user_128,91740004,group_062,COMPLETED,account_030,23.882500,0,0,0.933599
29572,user_128,91740008,group_062,COMPLETED,account_030,23.882222,0,0,0.933886
29575,user_128,91740012,group_062,COMPLETED,account_030,23.895556,0,0,0.938701
29530,user_128,91739952,group_062,COMPLETED,account_030,23.808889,0,0,0.906516


In [19]:
FindUnterutilizerSLURM(slurm_cleaned,17,0,10,0.95).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9204 entries, 75766 to 29500
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   User                      9204 non-null   object 
 1   JobID                     9204 non-null   object 
 2   Group                     9204 non-null   object 
 3   State                     9204 non-null   object 
 4   Account                   9204 non-null   object 
 5   UnderUtilizeTime (Hours)  9204 non-null   float64
 6   UnderUtilizeCPU           9204 non-null   object 
 7   UnderUtilizeNodes         9204 non-null   object 
 8   UnderUtilizeMemory        9130 non-null   float64
dtypes: float64(2), object(7)
memory usage: 719.1+ KB


/tmp/local/32730612/ipykernel_248495/2472876881.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['UnderUtilizeCPU'] = data.ReqCPUS - data.AllocCPUS
/tmp/local/32730612/ipykernel_248495/2472876881.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['UnderUtilizeNodes'] = data.ReqNodes - data.NNodes
/tmp/local/32730612/ipykernel_248495/2472876881.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [20]:
# future: group by users that underutilized, sort their max underutlize columns count; see the count of times they have underutilized
# each resources?

In [34]:
def FindUnterutilizerSLURM(data, time_threshold, cpu_threshold, nodes_threshold, memory_threshold):
    """
    Identifies underutilized SLURM job resources based on specified thresholds for time, CPU, nodes, and memory usage, 
    returning a DataFrame that highlights these jobs along with their underutilization percentage
    
    Parameters:
    :param data: DataFrame with SLURM job records.
    :param time_threshold: Minimum fraction of hours underutilization to be included.
    :param cpu_threshold: Minimum fraction of CPUs underutilization to be included.
    :param nodes_threshold: Minimum fraction of nodes underutilization to be included.
    :param memory_threshold: Minimum fraction (decimal, not percentage) of memory underutilization for inclusion.
    
    Returns:
    DataFrame with columns for 'User', 'JobID', 'Group', 'State', 'Account', and underutilized % resources.
    """
    
    # Calculate underutilized resources %
    data['UnderUtilizeCPU %'] = ( (data.ReqCPUS - data.AllocCPUS) / data.ReqCPUS )   
    data['UnderUtilizeNodes %'] = ( (data.ReqNodes - data.NNodes ) / data.ReqNodes) 
    data['UnderUtilizeTime (Hours) %'] = ( (data['Timelimit'] - data['Elapsed'])/pd.Timedelta(hours=1) ) / (data['Timelimit']/pd.Timedelta(hours=1))  
    data['UnderUtilizeMemory %'] = ((data['ReqMem_MB'] - data['MaxRSS_MB']) / data['ReqMem_MB'])
    
    # Apply thresholds and filter rows
    filtered_data = data.loc[
        (data['UnderUtilizeCPU %'] > cpu_threshold) |
        (data['UnderUtilizeNodes %'] > nodes_threshold) |
        (data['UnderUtilizeTime (Hours) %'] > time_threshold) |
        (data['UnderUtilizeMemory %'] > memory_threshold)
    ]
    
    return filtered_data[['User', 'JobID', 'Group', 'State', 'Account', 'UnderUtilizeTime (Hours) %', 'UnderUtilizeCPU %', 'UnderUtilizeNodes %', 'UnderUtilizeMemory %']]


In [36]:
Underutilized_df = FindUnterutilizerSLURM(slurm_cleaned,time_threshold=0.6, cpu_threshold=0.6, nodes_threshold=0.6, memory_threshold=0.95)
Underutilized_df

/tmp/local/32730612/ipykernel_248495/2503111120.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['UnderUtilizeCPU %'] = ( (data.ReqCPUS - data.AllocCPUS) / data.ReqCPUS )
/tmp/local/32730612/ipykernel_248495/2503111120.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['UnderUtilizeNodes %'] = ( (data.ReqNodes - data.NNodes ) / data.ReqNodes)
/tmp/local/32730612/ipykernel_248495/2503111120.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

,User,JobID,Group,State,Account,UnderUtilizeTime (Hours) %,UnderUtilizeCPU %,UnderUtilizeNodes %,UnderUtilizeMemory %
75766,user_316,91806820,group_009,COMPLETED,account_017,0.861404,0.0,0.0,0.965162
75880,user_316,91806820,group_009,FAILED,account_017,0.957895,0.0,0.0,0.999184
77026,user_316,91806820,group_009,FAILED,account_017,0.892982,0.0,0.0,0.965503
77281,user_316,91806820,group_009,COMPLETED,account_017,0.842105,0.0,0.0,0.951030
76963,user_316,91806820,group_009,FAILED,account_017,0.942982,0.0,0.0,0.972960
...,...,...,...,...,...,...,...,...,...
29569,user_128,91740004,group_062,COMPLETED,account_030,0.995104,0.0,0.0,0.933599
29572,user_128,91740008,group_062,COMPLETED,account_030,0.995093,0.0,0.0,0.933886
29575,user_128,91740012,group_062,COMPLETED,account_030,0.995648,0.0,0.0,0.938701
29530,user_128,91739952,group_062,COMPLETED,account_030,0.992037,0.0,0.0,0.906516


In [37]:
def AggregateUnderutilization(filtered_data):
    """
    Aggregates underutilization by user, calculating max underutilize percentage for each column,
    and counts the number of underutilization instances.

    Parameters:
    :param filtered_data: DataFrame after filtering underutilized jobs from FindUnterutilizerSLURM.

    Returns:
    DataFrame with 'User', 'MaxUnderUtilizeTime %', 'MaxUnderUtilizeCPU %', 'MaxUnderUtilizeNodes %', 
    'MaxUnderUtilizeMemory %', and 'UnderutilizationCount'.
    """
    
    # Aggregate max underutilization % per resource for each user
    agg_funcs = {
        'UnderUtilizeTime (Hours) %': 'max',
        'UnderUtilizeCPU %': 'max',
        'UnderUtilizeNodes %': 'max',
        'UnderUtilizeMemory %': 'max',
    }
    max_underutilization = filtered_data.groupby('User').agg(agg_funcs)
    
    # Count underutilization instances per user
    underutilization_count = filtered_data.groupby('User').size().rename('UnderutilizationCount')
    
    # Merge the two results
    aggregated_data = pd.merge(max_underutilization, underutilization_count, left_index=True, right_index=True)
    
    # Sort users by maximum underutilization count or a specific column
    sorted_aggregated_data = aggregated_data.sort_values(by='UnderutilizationCount', ascending=False)
    
    return sorted_aggregated_data


In [38]:
AggregateUnderutilization(Underutilized_df)

,UnderUtilizeTime (Hours) %,UnderUtilizeCPU %,UnderUtilizeNodes %,UnderUtilizeMemory %,UnderutilizationCount
User,,,,,
user_489,0.779149,0.0,0.0,0.978281,2713
user_454,0.979167,0.0,0.0,0.938050,1614
user_495,0.999084,0.0,0.0,0.998245,1474
user_624,0.994583,0.0,0.0,0.999920,875
user_690,0.994603,0.0,0.0,0.980957,859
...,...,...,...,...,...
user_178,0.599553,0.0,0.0,0.966022,1
user_373,0.331883,0.0,0.0,0.998120,1
user_429,0.627060,0.0,0.0,-0.047964,1


In [40]:
def AggregateUnderutilizationGroup(filtered_data):
    """
    Aggregates underutilization by user, calculating max underutilize percentage for each column,
    and counts the number of underutilization instances.

    Parameters:
    :param filtered_data: DataFrame after filtering underutilized jobs from FindUnterutilizerSLURM.

    Returns:
    DataFrame with 'User', 'MaxUnderUtilizeTime %', 'MaxUnderUtilizeCPU %', 'MaxUnderUtilizeNodes %', 
    'MaxUnderUtilizeMemory %', and 'UnderutilizationCount'.
    """
    
    # Aggregate max underutilization % per resource for each user
    agg_funcs = {
        'UnderUtilizeTime (Hours) %': 'max',
        'UnderUtilizeCPU %': 'max',
        'UnderUtilizeNodes %': 'max',
        'UnderUtilizeMemory %': 'max',
    }
    max_underutilization = filtered_data.groupby('Group').agg(agg_funcs)
    
    # Count underutilization instances per user
    underutilization_count = filtered_data.groupby('Group').size().rename('UnderutilizationCount')
    
    # Merge the two results
    aggregated_data = pd.merge(max_underutilization, underutilization_count, left_index=True, right_index=True)
    
    # Sort users by maximum underutilization count or a specific column
    sorted_aggregated_data = aggregated_data.sort_values(by='UnderutilizationCount', ascending=False)
    
    return sorted_aggregated_data

In [41]:
AggregateUnderutilizationGroup(Underutilized_df)

,UnderUtilizeTime (Hours) %,UnderUtilizeCPU %,UnderUtilizeNodes %,UnderUtilizeMemory %,UnderutilizationCount
Group,,,,,
group_078,0.779149,0.0,0.0,0.978281,2713
group_154,0.994583,0.0,0.0,0.999920,2491
group_009,0.996667,0.0,0.0,0.999866,1530
group_096,0.998194,0.0,0.0,0.999974,1509
group_140,0.999084,0.0,0.0,0.998245,1474
group_114,0.999238,0.0,0.0,0.998276,1074
group_082,0.994603,0.0,0.0,0.980957,859
group_062,0.999560,0.0,0.0,0.998924,536
group_004,0.967454,0.0,0.0,0.934547,312
